In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import shutil
import os
import csv
%matplotlib inline

In [2]:
DATA_PATH = './data'
IMAGE_SIZE = 224

### 1. generate object images with perspective

In [3]:
object_img = cv2.imread('source-data/dt.png', cv2.IMREAD_UNCHANGED)
object_img = cv2.resize(object_img, (IMAGE_SIZE, IMAGE_SIZE))

rows,cols,ch = object_img.shape

SMALL_RECT_LEFT = rows*2/6
SMALL_RECT_RIGHT = rows*4/6
SMALL_RECT_TOP = cols*2/6
SMALL_RECT_BOTTOM = cols*4/6

In [4]:
def get_image(path, num_image=100):
    if os.path.isdir(path):
        shutil.rmtree(path)
    os.mkdir(path)

    corners = []

    i = 0
    while i < num_image:
        # random all points
        left = np.random.randint(0, SMALL_RECT_LEFT, size=2)
        right = np.random.randint(SMALL_RECT_RIGHT, cols, size=2)
        top = np.random.randint(0, SMALL_RECT_TOP, size=2)
        bottom = np.random.randint(SMALL_RECT_BOTTOM, rows, size=2)

        # re-random if bottom-left point is over the diagonal
        # because image start from top-left so the y-axis should be opposite
        top_right_to_bottom_left_slope = -(top[1]-bottom[0])/(right[0]-left[1])
        bottom_right_to_bottom_left_slope = -(bottom[1]-bottom[0])/(right[1]-left[1])
        if bottom_right_to_bottom_left_slope > top_right_to_bottom_left_slope:
            print('re-random at ', path, i)
            continue

        # declare points
        pts1 = np.float32([[0,0],[cols,0],[0,rows],[cols, rows]])

        top_left = [left[0], top[0]]
        right_top = [right[0], top[1]]
        left_bottom = [left[1], bottom[0]]
        right_bottom = [right[1], bottom[1]]
        pts2 = np.float32([top_left, right_top, left_bottom, right_bottom])


        # transform
        perspectiveTransform = cv2.getPerspectiveTransform(pts1,pts2)
        wrapPerspective = cv2.warpPerspective(object_img, perspectiveTransform, (cols, rows))

        pathlib.Path(path).mkdir(parents=True, exist_ok=True)
        cv2.imwrite(f'{path}/object_image{i}.png',wrapPerspective)

        points = np.concatenate((top_left, right_top, left_bottom, right_bottom))
        corners.append(points)

        i+=1
    file = open(f'{path}/corners.csv', 'w')
    with file:
        writer = csv.writer(file)
        writer.writerows(corners)

In [5]:
get_image('./data/train', 1024)
get_image('./data/test', 128)
get_image('./data/val', 128)

### 2. Train

In [9]:
import csv
import math

import cv2
from keras.applications.mobilenet import MobileNet, _depthwise_conv_block
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import *
from keras.models import *
from keras.preprocessing.image import *
from keras.utils import Sequence

ALPHA = 0.25
# IMAGE_SIZE = 128 move to top

EPOCHS = 5000
PATIENCE = 100


class DataSequence(Sequence):

    def __load_images(self, dataset):
        out = []
        for file_name in dataset:
            im = cv2.resize(cv2.imread(file_name), (self.image_size, self.image_size))
            out.append(im)

        return np.array(out)

    def __init__(self, csv_file, image_path, image_size, batch_size=32, feature_scaling=False):
        self.csv_file = csv_file
        with open(self.csv_file, "r") as file:
            reader = csv.reader(file, delimiter=",")
            arr = list(reader)

        self.y = np.zeros((len(arr), 8))
        self.x = []
        self.image_size = image_size

        # for index, (path, class_id, width, height, x0, y0, x1, y1) in enumerate(arr):
        for index, coner_points in enumerate(arr):
            self.y[index] = np.array(coner_points)
            self.x.append(f'{image_path}/object_image{index}.png')

        self.batch_size = batch_size
        self.feature_scaling = feature_scaling
        if self.feature_scaling:
            dataset = self.__load_images(self.x)
            broadcast_shape = [1, 1, 1]
            broadcast_shape[2] = dataset.shape[3]

            self.mean = np.mean(dataset, axis=(0, 1, 2))
            self.mean = np.reshape(self.mean, broadcast_shape)
            self.std = np.std(dataset, axis=(0, 1, 2))
            self.std = np.reshape(self.std, broadcast_shape) + K.epsilon()

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        images = self.__load_images(batch_x).astype('float32')
        if self.feature_scaling:
            images -= self.mean
            images /= self.std

        return images, batch_y


def create_model(size, alpha):
    model_net = MobileNet(input_shape=(size, size, 3), include_top=False, alpha=alpha)
    x = _depthwise_conv_block(model_net.layers[-1].output, 1024, alpha, 1, block_id=14)
    x = MaxPooling2D(pool_size=(4, 4))(x)
    x = Conv2D(8, kernel_size=(1, 1), padding="same")(x)
    x = Reshape((8,))(x)
    
    # if I use flatten it will cause overfitting, I don't know why
#     x = Flatten()(x)
#     x = Dense(8)(x)

    return Model(inputs=model_net.input, outputs=x)


def train(model, epochs, image_size):
    train_datagen = DataSequence("./data/train/corners.csv", "./data/train", image_size)
    validation_datagen = DataSequence("./data/val/corners.csv", "./data/val", image_size)

    model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])
    checkpoint = ModelCheckpoint("model-{val_acc:.2f}.h5", monitor="val_acc", verbose=1, save_best_only=True,
                                 save_weights_only=True, mode="auto", period=1)
    stop = EarlyStopping(monitor="val_acc", patience=PATIENCE, mode="auto")

    model.fit_generator(train_datagen, steps_per_epoch=1028, epochs=epochs, validation_data=validation_datagen,
                        validation_steps=22, callbacks=[checkpoint, stop])


def main():
    model = create_model(IMAGE_SIZE, ALPHA)
    train(model, EPOCHS, IMAGE_SIZE)


if __name__ == "__main__":
    main()


Epoch 1/5000
1028/1028 [==============================] - 186s 181ms/step - loss: 879.2863 - acc: 0.6590 - val_loss: 21.8353 - val_acc: 0.8991

Epoch 00001: val_acc improved from -inf to 0.89915, saving model to model-0.90.h5
Epoch 2/5000
1028/1028 [==============================] - 178s 173ms/step - loss: 4.8065 - acc: 0.9389 - val_loss: 10.7575 - val_acc: 0.9233

Epoch 00002: val_acc improved from 0.89915 to 0.92330, saving model to model-0.92.h5
Epoch 3/5000
1028/1028 [==============================] - 178s 173ms/step - loss: 1.8900 - acc: 0.9646 - val_loss: 8.6463 - val_acc: 0.9531

Epoch 00003: val_acc improved from 0.92330 to 0.95312, saving model to model-0.95.h5
Epoch 4/5000
1028/1028 [==============================] - 178s 173ms/step - loss: 1.6226 - acc: 0.9630 - val_loss: 6.5241 - val_acc: 0.9602

Epoch 00004: val_acc improved from 0.95312 to 0.96023, saving model to model-0.96.h5
Epoch 5/5000
1028/1028 [==============================] - 178s 173ms/step - loss: 1.2680 - acc:

KeyboardInterrupt: 